# 10. 한 걸음 더

지금까지 강화 학습에 대해 많은 것을 배웠습니다. 이 책의 전반부에서는 강화 학습의 기초를 배웠고, 후반부에서는 강화 학습에 딥러닝을 결합한 중요한 기법들을 살펴보았습니다. 이번 장에서는 현대적인 심층 강화 학습으로 한 걸음 더 들어간 알고리즘들을 소개하고자 합니다. 먼저 심층 강화 학습의 전체 그림을 보여드리고 이어서 정책 경사법 계열의 알고리즘들을, 그 다음에는 DQN 계열의 알고리즘들을 살펴보겠습니다.

또한 심층 강화 학습에서 중요한 연구 사례도 소개합니다. 바둑과 장기 같은 보드 게임, 로봇 제어, 반도체 설계 등 사회적으로 큰 성과를 거둔 사례들을 준비했습니다. 마지막으로 심층 강 화학습의 가능성과 풀어야 할 숙제를 논의하며 이 책을 마무리합니다.

## 10.1 심층 강화 학습 알고리즘 분류

심층 강화 학습 알고리즘들은 크게 [그림 10-1]처럼 분류할 수 있습니다. 분류 방법은 OpenAI의 문헌[20]을 참고했습니다.

<p align="center"><img src="./images/fig_10_01.png" width=800></p>

먼저 가장 큰 분류 기준은 환경 모델, 즉 상태 전이 함수 p(s'| s, a)와 보상 함수 r(s, a, s')의 사용 여부입니다. 환경 모델을 사용하면 모델 기반 기법 model-based method, 사용하지 않으면 모델 프리 기법 model-free method 이라고 합니다.

모델 기반 기법은 다시 환경 모델이 주어지는 경우와 환경 모델을 학습하는 경우로 나눌 수 있 습니다. 

환경 모델이 주어지면 에이전트는 행동 없이 계획 planning 만으로 문제를 해결할 수 있습 니다. 앞서 4장에서도 환경 모델을 이용하여 동적 프로그래밍으로 문제를 풀었습니다. 바둑이나 장기 같은 보드 게임도 환경 모델이 알려져 있기 때문에 이 분류의 기법을 활용할 수 있습니다. 유명한 알고리즘으로는 알파고AlphaGo와 알파제로AlphaZero 등이 있습니다 (자세한 내용은 10.4.1절에서 설명합니다).

환경 모델이 주어지지 않으면 환경에서 얻은 경험을 토대로 환경 모델을 학습하는 방법을 생각해볼 수 있습니다. 학습한 환경 모델은 계획 수립뿐 아니라 정책의 평가와 개선에도 활용할 수 있습니다. 이 분류에 속하는 기법으로는 World Models[21]와 MBVEMoc [22] Model-Based Value Estimation 등이 있습니다. 현재 활발히 연구 중인 분야로, 앞으로 인간과 같은 범용 인공지능을 구현하는 데에도 유용하게 활용될 것으로 기대되고 있습니다.

> CAUTION_ 환경 모델을 학습하는 방법에는 몇 가지 문제가 있습니다. 가장 큰 문제는 에이전트가 환경 모델이 생성하는 샘플 데이터의 일부만 얻을 수 있다는 점입니다. 학습한 환경 모델과 실제 환경 사이에 괴리(편향)가 생길 수 있다는 이야기죠. 그 결과 에이전트가 학습한 모델에서는 적합했던 행동이 실제 환경에서 는 바람직하지 않은 행동일 수 있습니다.

환경 모델을 학습하는 기법들은 잠재력이 크지만, 현재까지는 모델 프리 기법들이 더 많은 성과를 거두고 있습니다. 이 책에서 소개하는 기법들도 모델 프리입니다. 모델 프리 기법은 크게 정책 기반 기법과 가치 기반 기법, 그리고 이 둘을 모두 갖춘 기법으로 분류할 수 있습니다.

정책 기반 기법에는 정책 경사법과 REINFORCE가 해당되며, 행위자-비평자는 '정책 기반이자 가치 기반 기법입니다(9장 참고). 10.2절에서 정책 경사법을 발전시킨 알고리즘들을 소개 합니다.

가치 기반 기법은 가치 함수를 모델링하여 학습하는 기법입니다. 그중 가장 중요한 기법은 DQN입니다. 10.3절에서 DQN을 발전시킨 알고리즘들을 소개합니다.

## 10.2 정책 경사법 계열의 고급 알고리즘

9장에서는 REINFORCE와 행위자-비평자 등의 정책 경사법 알고리즘에 대해 설명했습니다. 이번 절에서는 같은 계열에서 더욱 고도화된 알고리즘들을 소개합니다. 소개할 알고리즘은 다음과 같습니다.

- A3C, A2C (분산학습 알고리즘)
- DDPG (결정적 정책을 따르는 알고리즘)
- TRPO, PPO (목적 함수에 제약을 추가하는 알고리즘)

알고리즘의 특징에 따라 이와 같이 세 그룹으로 나눴습니다. A3C와 A2C부터 만나보겠습니다.

### 10.2.1 A3C, A2C

A3C[23]는 'Asynchronous Advantage Actor-Critic'의 약자입니다. 이름에 A가 세 개, C가 한 개여서 A3C라고 하죠. A3C의 특징은 Asynchronous, 즉 '비동기'라는 점입니다. 여기서 말하는 비동기란 여러 에이전트가 병렬로 행동하며 비동기적으로 매개변수를 갱신한다는 뜻입 니다.

> NOTE_ 컴퓨터 과학에서는 '동시concurrent'와 '병렬parallel'을 다른 의미로 씁니다. '동시'는 한 번에 하나의 작업만 처리합니다. 하지만 여러 작업 사이를 빠르게 전환하여, 마치 작업 여러 개를 동시에 수행하는 것처럼 보이게 하는 기술입니다. 한편 '병렬은 물리적으로 다른 장소(CPU 코어나 GPU 코어)에서 여러 작업을 실제로 동시에 처리합니다.

A3C는 신경망으로 모델링한 행위자-비평자를 이용하여 정책을 학습합니다. 그리고 [그림10-2]와 같이 하나의 전역 신경망과 여러 개의 지역 신경망을 사용합니다.

<p align="center"><img src="./images/fig_10_02.png" width=600></p>

지역 신경망은 각자의 환경에서 독립적으로 플레이하며 학습합니다. 그리고 학습 결과인 기울기를 전역 신경망에 보냅니다. 전역 신경망은 여러 지역 신경망에서 보내온 기울기를 이용해 비동기적으로 가중치 매개변수를 갱신합니다. 이렇게 전역 신경망의 가중치 매개변수를 갱신 하는 동안 주기적으로 전역 신경망과 지역 신경망의 가중치 매개변수를 동기화합니다.

A3C의 장점은 여러 에이전트를 병렬로 행동하게 하여 학습 속도를 높인다는 것입니다. 게다가 에이전트들이 독립적으로 행동(탐색)하기 때문에 한층 다양한 데이터를 얻을 수 있습니다. 그 덕분에 학습 데이터의 전체적인 상관관계를 약화시킬 수 있어 학습이 더욱 안정적으로 이루어 집니다.

> NOTE_ DQN에서는 학습 데이터의 상관관계를 약화시키기 위해 경험 재생 기법을 사용했습니다. 경험 재생은 경험 데이터를 버퍼에 저장해두고, 거기서 무작위로 여러 개를 골라내는 기법입니다. 하지만 경험 재생은 온정책 방식으로는 사용할 수 없습니다. 메모리 사용량과 계산량이 커진다는 것도 단점입니다.

병렬 처리는 온정책 방식에서도 사용할 수 있는 범용 아이디어입니다. 여러 에이전트를 병렬로 움직여 (경험 재생에 의존하지 않고도) 데이터의 상관관계를 약화시킬 수 있죠. 실제로 A3C 논문 발표 이후 많은 연구에서 병렬 처리가 유행처럼 번지고 있습니다.

또한 A3C의 행위자-비평자는 신경망의 가중치를 공유합니다. [그림 10-3]과 같이 하나의 신경망에서 가중치를 공유하여 정책과 가치 함수를 출력합니다. 정책과 가치 함수 모두 입력에 가까운 층은 가중치가 비슷할 것이라서 이처럼 신경망 사이에 매개변수를 공유하는 구조가 효과가 있습니다.

<p align="center"><img src="./images/fig_10_03.png" width=600></p>

다음으로 A2C[23]를 보겠습니다. A2C는 매개변수를 (비동기가 아니라) 동기식으로 갱신합니다. A3C에서 첫 번째 A (Asynchronous)가 빠진 이름이죠. A2C는 [그림 10-4]와 같은 구성으로 구현할 수 있습니다.

<p align="center"><img src="./images/fig_10_04.png" width=600></p>

그림과 같이 각각의 환경에서 에이전트가 독립적으로 동작합니다. 따라서 시간에서의 상태는 환경마다 다릅니다. 그리고 시간 t에서 각 환경의 상태를 (동기화하여) 배치로 묶어 신경망을 학습시킵니다. 이때 신경망의 출력 (정책의 확률 분포)에서 다음 행동을 샘플링하며, 샘플링된 행동을 각 환경에 전달합니다.

실험 결과 A2C 방식의 동기식 갱신 성능도 A3C보다 떨어지지 않았습니다. 그런데 구현하기는 A2C가 더 쉽고 GPU 같은 컴퓨트 자원을 더 효율적으로 사용할 수 있습니다. 이런 장점 때문에 실무에서는 A2C를 더 많이 이용합니다.

> NOTE_ A3C는 신경망을 환경별로 실행해야 합니다. 따라서 2개의 환경을 준비한다면 이상적으로는 GPU도 2개 준비해야 합니다. 반면 A2C에서는 신경망을 실행하는 부분이 하나로 통합되어 있어서 GPU 하나로 계산할 수 있습니다.

### 10.2.2 DDPG

정책을 직접 모델링하는 정책 경사법 같은 기법은 행동 공간이 연속적인 문제에도 활용할 수 있습니다. 예를 들어 [그림 10-5]와 같이 정책을 모델링한 신경망이 '정규분포의 평균'을 출력 하도록 설계할 수 있습니다. 실제 행동은 이 정규분포에서 샘플링하여 얻습니다.

<p align="center"><img src="./images/fig_10_05.png" width=600></p>

DDPG[24]는 Deep Deterministic Policy Gradient method (심층 결정적 정책 경사법)의 약자입니다. 이름이 말해주듯 연속적인 행동 공간에서의 문제에 맞춰 설계된 알고리즘입니다. [그림 10-6]과 같이 이 알고리즘의 신경망은 행동을 연속적인 값으로 직접 출력합니다.

<p align="center"><img src="./images/fig_10_06.png" width=400></p>

DDPG의 정책은 특정 상태 s를 입력하면 행동 a가 고유하게 결정되기 때문에 결정적 정책입 니다. DDPG에서는 이 결정적 정책을 DQN에 통합합니다. 정책을 나타내는 신경망을 $\mu_\theta(s)$ 라 하고, DQN의 Q 함수를 나타내는 신경망을 $Q_\phi(s)$라 하며, $\theta$와 $\phi$(파이)는 각 신경망의 매개변수입니다. 이 때 DDPG는 다음 두 가지 학습 과정을 거쳐 매개변수를 갱신합니다.

1. Q 함수의 출력이 커지도록 정책$\mu_\theta(s)$의 매개변수 $\theta$를 갱신
2. DQN에서 수행하는 Q러닝을 통해 Q 함수 $Q_\phi(s)$의 매개변수 $\phi$를 갱신

첫 번째 학습과정부터 알아보죠. 첫 번째 학습은 [그림 10-7]과 같이 두 개의 신경망을 조합하여 Q 함수의 출력이 최대가 되도록 결정적 정책$\mu_\theta(s)$의 매개변수 $\theta$를 갱신하는 것입니다.

<p align="center"><img src="./images/fig_10_07.png" width=600></p>

이 그림에서 중요한 점은 $\mu_\theta(s)$가 출력하는 행동 a가 연속적인 값이고, 이 출력 a가 그대로 $Q_\phi(s)$의 입력이 된다는 점입니다. 이렇게 연결된 두 신경망에서 역전파를 수행하며, 그 결과로 기울기 $▽_\theta(q)$가 구해지고(q는 Q 함수의 출력) 기울기 $▽_\theta(q)$를 이용해 매개변수를 갱신할 수 있습니다.

> NOTE_ [그림 10-7]에서 만약 행동을 확률적 정책으로 샘플링했다면 역전파가 샘플링을 하는 지점에서 멈춥니다. 샘플링 이후로는 기울기가 0으로만 전달된다는 이야기입니다. 이렇게 되면 정책의 매개변수를 갱신 할 수 없습니다.

두 번째 학습은 DQN으로 하는 Q러닝입니다. 방법은 8장에서 설명했습니다. 다만 이번에는 결정적 정책 $\mu_\theta(s)$를 사용하여 계산 효율을 높일 수 있습니다.

DQN에서 Q 함수의 갱신은 $Q_\phi(S_t, A_t)$의 값이 $R_t + \gamma \displaystyle\max_{a} Q_\phi(S_{t+1}, a)$가 되도록 (또는 근접하도록)하는 것이었습니다. 그런데 앞서 첫 번째 학습을 통해 정책 $\mu_\theta(s)$는 Q함수가 커지는 행동을 출력합니다. 따라서 다음 근사를 적용할 수 있습니다.

$$\displaystyle\max_{a} Q_\phi(s, a) \simeq Q_\phi(s, \mu_\theta(s))$$

최댓값을 구하는 $\displaystyle\max_{a}$는 일반적으로 계산량이 많습니다. 그런데 DDPG에서는 $\displaystyle\max_{a} Q_\phi(s, a)$ 계산을 $Q_\phi(s, \mu_\theta(s))$라는 두 개의 신경망 순전파(그림 10-7)로 대체합니다. 이렇게 계산을 단순화하여 학습 효율을 높이는 것입니다.

> NOTE_ DDPG는 ‘소프트 목표'와 '탐색 노이즈'라는 아이디어도 활용합니다. 소프트 목표 soft target는 DQN의 '목표 신경망'을 '부드럽게' 만든 기법입니다. 목표 신경망의 매개변수를 학습 중인 매개변수와 주기적으로 동기화하는 대신, '매번 학습 중인 매개변수의 방향에 가까워지도록 갱신하는 것이죠. 탐색 노이즈 exploration noise는 결정적 정책에 노이즈를 넣어 행동에 무작위성을 주입하는 기법입니다. 자세한 내용은 DDPG 논문 [24]을 참고하기 바랍니다.

### 10.2.3 TRPO, PPO

정책 경사에서는 정책을 신경망으로 모델링하고 기울기 기반으로 매개변수를 갱신합니다. 정책 경사법은 기울기로 매개변수의 갱신 '방향'은 알 수 있지만, 얼마만큼 갱신해야 좋은가 를 뜻하는 '갱신 폭은 알 수 없다는 단점이 있습니다. 폭이 너무 넓으면 정책이 나빠지고, 반대로 너무 좁으면 학습이 거의 진행되지 않습니다. 이 문제를 해결한 것이 **TRPO** Trust Region Policy Optimization [25]입니다. '신뢰 영역 정책 최적화'라고 번역할 수 있습니다. 이름에서 알 수 있듯이 신뢰할 수 있는 영역 안에서, 즉 적절한 갱신 폭으로 정책을 최적화할 수 있습니다.

두 확률 분포가 얼마나 유사한지를 측정하는 지표로 **쿨백-라이블러 발산** Kullback-Leibler Divergence (KLD)이 있습니다. TRPO에서는 정책 갱신 전후의 쿨백-라이블러 발산을 지표로 삼아, 그 값이 임곗값을 넘지 않아야 한다는 제약을 부과합니다. 즉, 쿨백-라이블러 발산 제약이 걸린 상태에서 목적 함수를 최대화하는 문제로 보는 것입니다. 이 제약 덕분에 적절한 갱신 폭을 구할 수 있습니다.

> NOTE_ TRPO의 세부 내용(특히 수학적 도출)은 복잡하기 때문에 자세한 설명은 생략합니다. 중요한 점은 기울기 갱신 폭을 적절하게 관리하기 위해, 한 번의 갱신으로 정책이 너무 크게 변하지 않도록 제약을 부과한다는 것입니다. 이 제약 안에서 학습을 진행하는 것이죠. 이후는 제약이 걸린 최적화 문제이며, 수학적으로 해결할 수 있습니다.

TRPO에서 제약이 걸린 최적화 문제를 풀려면 헤세 행렬 hessian matrix이라는 이차 미분 계산이 필요합니다. 헤세 행렬은 계산량이 많아 병목을 일으킵니다. 이 문제를 개선한 방법이 ppoProximal Policy Optimization [26]입니다. PPO는 TRPO를 단순화한 기법으로, 계산량을 줄이면 서도 성능은 TRPO와 비슷하여 실무에서 많이 활용합니다.

## 10.3 DQN 계열의 고급 알고리즘

DQN은 딥러닝에서 가장 중요한 알고리즘입니다. 현재도 DQN 기반의 새로운 확장 기법들이 많이 제안되고 있지요. 이번 절에서는 DQN을 기초로 발전시킨 알고리즘 중 특히 중요한 몇 가지를 소개합니다. 살펴볼 알고리즘은 [그림 10-8]과 같습니다.

<p align="center"><img src="./images/fig_10_08.png" width=600></p>


그림과 같이 심층 강화 학습은 DQN을 토대로 꾸준히 개선하는 형태로 발전해왔습니다. 그 중 다음 세 기법은 8.4절에서 이미 설명했습니다.

- Double DQN
- 우선순위 경험 재생(Prioritized Experience Replay) 
- Dueling DQN

이번 절에서는 나머지 기법들을 만나보겠습니다.

### 10.3.1 범주형 DQN

범주형 DQN Categorical DQN [27] 기법들부터 알아보겠습니다. 이미 보았듯이 Q 함수의 수식은 다 음과 같습니다.

$$Q_\pi(s,a)=E_\pi[G_t|S_t=s, A_t = a]$$

[그림 10-9]와 같이 확률적 사건인 수익 $G_t$를 기댓값이라는 하나의 값으로 표현하는 것이 Q 함수의 특징입니다.

<p align="center"><img src="./images/fig_10_09.png" width=600></p>

DQN에서는 (더 나아가 Q 러닝에서는) Q 함수, 즉 기댓값으로 표현되는 값을 학습합니다. 이를 발전시켜 Q 함수라는 기댓값이 아니라 '분포'를 학습시키자는 아이디어가 있습니다. 이 아이디어를 분포 강화 학습 distributional reinforcement learning이라고 합니다. 분포 강화 학습에서는 수익의 확률 분포인 $Z_\pi(s, a)$를 학습합니다.

범주형 DQN은 바로 이 분포 강화 학습을 기반으로 합니다. 여기서 '범주형'이란 [그림 10-10]과 같이 범주형 분포로 모델링한다는 뜻입니다.

<p align="center"><img src="./images/fig_10_10.png" width=400></p>

범주형 분포는 여러 범주(이산 값) 중 어느 범주에 속할 것인지에 대한 확률분포입니다. [그림 10-10]과 같이 수익이 취하는 값이 몇 개의 영역(빈bin)으로 나뉘고, 각 빈에 들어갈 확률이 범주형 분포로 모델링됩니다.

범주형 DQN에서는 수익을 범주형 분포로 모델링하고 그 '분포의 형태'를 학습합니다. 이를 위 해 범주형 분포 버전의 벨만 방정식을 도출하고, 그 방정식을 이용하여 범주형 분포를 갱신합 니다. 참고로 범주형 분포의 빈이 51개일 때 '아타리' 과제에서 성능이 가장 좋았기 때문에 범 주형 DQN 'C51'이라고도 부릅니다.

### 10.3.2 Noisy Network

DQN에서는 ε-탐욕 정책으로 행동을 결정합니다. ε의 확률로 무작위 행동을 선택하고, 나머 지 1-ε의 확률로 탐욕 행동 (Q 함수가 가장 큰 행동)을 선택하죠. 실전에서는 대체로 에피소드가 진행될수록 값을 조금씩 낮추도록 스케줄링합니다. 여기서 문제는 ε값인데, ε은 하이퍼 파라미터라서 어떻게 설정하느냐에 따라 최종 정확도가 크게 달라질 수 있습니다. 하지만 ε값 의 후보는 매우 다양합니다.

이러한 설정 문제를 해결하기 위해 Noisy Network[28]가 제안되었습니다. Noisy Network는 신경망에 무작위성을 도입합니다. 그 덕분에 행동을 (ε-탐욕 정책이 아닌) 탐욕 정책에 따라 선택할 수 있습니다. 정확하게는 출력 쪽의 완전 연결 계층에서 '노이즈가 들어간 완전 연결 계층'을 사용합니다. '노이즈가 들어간 완전 연결 계층'에서 가중치는 정규분포의 평균과 분산으로 모델링되며 (순전파할 때마다) 가중치가 정규분포에서 샘플링됩니다. 이렇게 하면 순전파할 때마다 무작위성이 스며들어 최종 출력이 달라집니다.

### 10.3.3 레인보우

지금까지 다양한 DQN 확장 알고리즘을 소개했습니다. 그리고 이 모든 것을 결합한 기법이 바로 레인보우 Rainbow [29]입니다. 레인보우는 기존 DQN에 다음과 같은 기법들을 모두 조합하여 사용합니다.

- Double DQN
- 우선순위 경험 재생
- Dueling DQN
- 범주형 DQN
- Noisy Network

[그림 10-11]은 아타리 게임에서 레인보우와 그 외 기법들의 성능을 측정한 결과입니다.

<p align="center"><img src="./images/fig_10_11.png" width=500></p>


그림의 가로축은 학습에 사용한 이미지 개수입니다(표기된 수치 × 100만). 세로축은 일반인과 비교하여 정규화한 점수이며, 높을수록 성능이 좋다는 뜻입니다. 이 그림과 같이 레인보우는 다른 기법들에 비해 성능이 비약적으로 높음을 알 수 있습니다.

### 10.3.4 레인보우 이후의 발전된 알고리즘

레인보우 이후 CPU/GPU를 이용한 여러 분산 병렬 학습이 큰 성과를 올렸습니다. 이를 **분산 강화 학습**이라고도 하며, 실행 환경을 여러 개 준비하여 학습을 병렬로 진행합니다. 분산 강화 학습으로 유명한 기법이 **Ape-X**[30]입니다. Ape-X는 레인보우를 기반으로 여러 개의 에이전트를 각각의 CPU에서 독립적으로 행동시킵니다. 이때 에이전트들의 탐색 비중인 ε을 모두 다르게 설정하여 다양한 경험 데이터를 수집합니다. 이처럼 분산 병렬화를 통해 학습을 빠르게 진행하는 동시에 경험 데이터를 다양하게 얻어 성능을 높였습니다.

**R2D2**[31]는 Ape-X를 더욱 개선한 기법입니다. R2D2는 Ape-X에 더해 시계열 데이터를 처리하는 순환 신경망(RNN)을 사용했습니다(정확하게는 LSTM 사용). 간단한 아이디어지만 RNN으로 학습하기 위해 많은 노력을 기울여 Ape-X의 성능을 한층 높이는 데 성공했습니다. 참고로 R2D2의 이름은 Recurrent (순환)와 Replay (경험 재생)에서 'R' 두 개를 가져오고, Distributed(분산)와 Deep Q-Network (DQN)에서 'D' 두 개를 가져와서 만든 이름입니다(물론 영화 <스타워즈>의 유명 캐릭터 이름에서 따온 것이기도 합니다).

다음은 R2D2를 더욱 발전시킨 **NGU**[32]입니다. NGU는 'Never Give Up (절대 포기하지 마)' 의 약자입니다. NGU는 R2D2의 토대에 내적 보상 intrinsic reward 메커니즘을 추가하여 어려운 과제, 특히 보상이 적은 과제에서도 탐색을 포기하지 않도록 했습니다. 알고리즘 이름은 이러한 특성에서 유래했습니다. 내적 보상은 상태 전이가 예상과 다를수록, 즉 얼마나 놀랐는가'에 따 라 스스로 보상을 더해주는 기법입니다. 보상이 0에 가까운 희박한 작업에서 내적 보상은 (보상의 크기가 아니라) '호기심'에 따라 행동하도록 유도합니다. 이 과정에서 (잘만 하면) 보상을 극대화하는 방법을 찾을 수 있습니다.

> NOTE_ 아이들, 특히 어린아이는 새롭고 놀라운 경험을 찾아 놀기를 좋아합니다. 확실한 목적을 가지고 학습하기보다는 호기심이 이끄는 대로 행동합니다. 내적 보상의 목표는 바로 이처럼 호기심을 좇는 행동을 에이전트에 주입하는 것입니다.

마지막으로 만나볼 기법은 **Agent57**[33]입니다. NGU를 발전시킨 기법이죠. 중요한 특성은 내적 보상 메커니즘을 개선하고 '메타 컨트롤러'라는 구조를 사용하여 에이전트들에 할당되는 정책을 유연하게 배분했다는 점입니다. 아타리, 정확하게는 '아타리 2600'에는 게임이 모두 '57' 개가 있습니다. Agent57은 이 모든 게임에서 사람보다 우수한 성적을 거두는 데 성공했습니 다. 강화 학습 알고리즘으로서는 처음 이룬 쾌거였습니다.

## 10.4 사례 연구

딥러닝 사례로는 비디오 게임이나 바둑과 같은 보드 게임이 유명하지만 그 외에도 로봇 제어, 자율주행, 의료, 금융, 바이오 등 다양한 분야에서 성과를 거두고 있습니다. 이번 절에서는 딥러닝이 어떤 용도로 활용되는지를 보여드릴 겸, 유명한 사례를 몇 가지 소개하겠습니다.

### 10.4.1 보드 게임

바둑, 장기, 오셀로 등의 보드 게임은 다음과 같은 특성을 지니고 있습니다.

- 보드의 모든 정보를 알 수 있음(완전 정보)
- 한쪽이 승리하면 다른 쪽은 패배함(제로섬 zero-sum)
- 상태 전이에 우연이라는 요소가 없음(결정적)

이런 성질을 지닌 게임에서 중요한 것은 '수 읽기'입니다. “내가 이 수를 두면 상대가 저 수를 둘 것이고, 그러면 나는 이 수로 받아칠 것이다"처럼 앞으로 일어날 일을 다양하게 예측해볼수록 더 좋은 수를 찾을 수 있습니다. 보드 게임에서 '수 읽기'는 [그림 10-12]처럼 트리 구조로 표현할 수 있습니다.

<p align="center"><img src="./images/fig_10_12.png" width=500></p>


그림과 같이 보드를 노드로, 다음 수를 에지(화살표)로 표현한 것을 게임 트리 game tree라고 합니 다. 게임 트리를 모두 펼칠 수 있다면 가능한 모든 결과가 드러나기 때문에 최선의 수를 찾을 수 있습니다. 하지만 바둑이나 장기는 가능한 상태 (돌이나 말을 배치하는 패턴)가 천문학적으로 많아서 게임 트리를 모두 전개하기가 현실적으로 불가능합니다. 따라서 문제를 풀려면 게임 트리를 효율적으로 탐색해야 합니다.

> NOTE_ 게임 트리에서 가능한 상태(노드)의 수는 보드 게임마다 다릅니다. 오셀로는 1058개, 체스는 10123개, 장기는 10226개, 바둑은 10400개입니다.

**몬테카를로 트리 탐색** Monte Carlo Tree Search (MCTS)은 트리의 전개를 몬테카를로법으로 근사하는 기법입니다. 예를 들어 돌들이 특정 형태로 놓여 있는 보드 상태가 얼마나 좋은지 평가할 때, 돌을 무작위로 두는 두 플레이어에게 승패가 결정될 때까지 계속 두게 합니다. 이러한 시도(특정상태의 보드에서 시작하는 게임)를 수차례 반복하여, 그 승률로 해당 보드 상태가 얼마나 좋은가'를 근사적으로 나타냅니다. 이처럼 승패가 결정될 때까지 무작위로 게임을 진행시키는 일을 플레이아웃playout 또는 롤아웃rollout이라고 합니다.

몬테카를로트리 탐색은 현재의 보드 상태에서 가능한 모든 수를 각각 평가하고, 그 결과를 토 대로 다음 수를 결정하는 흐름으로 진행합니다. 그리고 일반적으로는 이 기본 아이디어에 더하여 유망해 보이는 보드 상태로 전개한 후 평가한 결과를 피드백하는 메커니즘이 보강됩니다. 몬테카를로 트리 탐색에 대한 자세한 내용은 해당 논문 [34]을 참고하기 바랍니다.

**알파고**

알파고 AlphaGo [35]는 몬테카를로 트리 탐색에 심층 강화 학습을 결합한 기법입니다. 2016년 세계 최고의 바둑 기사에게 승리를 거두며 세상을 놀라게 한 주인공이죠.

알파고에는 두 개의 신경망이 사용되는데, 하나는 현재 바둑판에서 이길 확률을 평가하는 Value 신경망이고 다른 하나는 정책을 나타내는 Policy 신경망입니다. Policy 신경망은 다음에 둘 수를 확률로 출력합니다. 예를 들어 (1, 1) 위치에 둘 확률은 2.4%, (1, 2) 위치에 둘 확 률은 0.2%와 같은 식이죠. 이 두 신경망으로 몬테카를로 트리 탐색을 더 정밀하게 수행할 수 있게 되었습니다.

알파고는 인간의 기보 데이터를 이용해 두 신경망을 학습시켰습니다. 그런 다음 셀프 플레이 self play 형태로 대국을 반복하고 여기서 수집한 경험 데이터를 사용해 학습을 더욱 강화했습니다.

> NOTE_ 셀프 플레이는 강화 학습의 틀 안에서 설명할 수 있습니다. 바둑을 두는 '에이전트'가 있고 에이전트의 복제체인 대전 상대가 '환경' 역할을 합니다. 이 둘이 상호작용하면서 최종적으로 승리 혹은 패배라는 보상을 얻습니다. 이처럼 환경 속에서 상호작용하며 데이터를 수집한다는 점에서 알파고가 활용한 방식은 강화 학습이라고 말할 수 있습니다.

**알파고 제로**

알파고는 인간의 기보를 학습 데이터로 사용했습니다. 하지만 알파고 제로AlphaGo Zero [36]는 학습 데이터를 전혀 사용하지 않고 셀프 플레이를 통한 강화 학습만으로 학습했습니다. 또한 알파고 에서 사용하던 '도메인 지식(바둑 규칙)'을 이용하지 않은 점도 큰 특징입니다. 이 외에도 다음 과 같은 점을 개선했습니다.

- Policy 신경망과 Value 신경망을 하나의 신경망으로 표현
- 몬테카를로트리 탐색으로 플레이아웃하지 않고, 신경망의 출력만으로 각 노드(상태)를 평가

이처럼 기존 알파고보다 단순하고 범용적으로 개선했습니다. 더구나 인간이 쌓아둔 기보 데이터 없이 강화 학습만으로 학습했음에도 놀랍게도 기보 데이터를 활용한 알파고를 압도하는 성능을 보여줬습니다. 실제로 알파고와의 대국에서 100대 0으로 압승을 거뒀습니다.

> NOTE_ 인간의 지식을 활용하는 지도 학습이 도달할 수 있는 수준은 학습 데이터에 반영된 인간의 실력까 지입니다. 한편 강화 학습은 스스로 새로운 경험을 쌓아가며 학습하기 때문에 이런 한계가 없습니다. 이론적 으로는 인간을 뛰어넘을 수 있는 것이죠. 실제로 알파고 제로는 인간의 수준을 한참 넘어섰습니다. 그래서 알 파고제로 또한 강화 학습의 가능성을 보여준 중요한 연구입니다.

**알파제로**

알파제로 AlphaZero [37]는 알파고 제로를 미세하게 조정한 것이지만 거의 같은 알고리즘을 이용합니다. 그럼에도 바둑뿐 아니라 체스와 장기까지 둘 수 있습니다. 보드 게임의 종류에 상관없는 범용 알고리즘으로 진화한 것입니다.

### 10.4.2 로봇 제어

딥러닝은 로봇 제어와 같은 시스템에도 활용됩니다. 하나의 예로, 구글은 로봇 팔이 다양한 물건을 잡을 수 있도록 학습시키는 데 성공했습니다. [38] [그림 10-13]과 같이 로봇은 위쪽에 장 착된 카메라가 보내주는 영상을 해석하여 어떻게 행동할지를 결정합니다. 행동의 결과로 물체 를 잡으면 성공, 잡지 못하면 실패라는 보상을 줍니다. 이처럼 강화 학습의 틀 안에서 학습을 진행합니다.

<p align="center"><img src="./images/fig_10_13.png" width=500></p>

필요한 데이터는 그림과 같이 로봇 일곱 대를 수개월 동안 실제 운용하여 수집했습니다. 이때 QT-Opt라는 강화 학습 알고리즘을 이용했습니다. QT-Opt는 Q 러닝 기반이기 때문에 오 프-정책 기법입니다. 과거에 얻은 경험 데이터도 활용한다는 뜻이죠. 로봇을 실제로 운용해야 하는 환경이라면 데이터 수집 비용이 많이 들기 때문에 과거 데이터를 활용할 수 있다는 건 커 다란 장점입니다.

논문에 따르면 QT-Opt는 미지의 물건도 96% 확률로 잡는 데 성공하여, 구글이 이전에 개발 한 지도 학습 방식보다 실패 확률을 1/5 이하로 줄였다고 합니다.

### 10.4.3 NAS

딥러닝의 아키텍처(신경망 구조)는 보통 사람이 직접 설계합니다. 뛰어난 아키텍처를 설계하 기 위해서는 경험이 필요하여, 수많은 시행착오를 거쳐야 합니다. 그런데 최근에는 최적의 아키텍처를 컴퓨터가 자동으로 설계하는 연구[39]가 활발히 진행되고 있습니다. 흔히들 **NAS** Neural Architecture Search라고 부르는 분야입니다.

NAS를 수행하는 방법에는 베이지안 최적화 Bayesian optimization와 유전 프로그래밍 genetic programming 등 여러 가지가 있습니다. 그중 유력한 후보가 강화 학습입니다. 붐을 일으킨 주인공은「Neural Architecture Search with Reinforcement Learning」 논문[40  ]입니다.  

이 논문에서는 강화 학습으로 신경망 구조를 자동으로 최적화하여 사람이 설계한 것 이상의 아키텍처 를 발견하는 데 성공했습니다. 어떻게 해냈는지를 간략하게 알아보겠습니다.

핵심 아이디어는 신경망 아키텍처를 '텍스트'로 표현할 수 있다는 점에 주목한 것입니다. 예를 들어 신경망의 아키텍처를 다음 예와 같은 텍스트로 정의할 수 있습니다(가상 포맷입니다).

<p align="center"><img src="./images/fig_10_13_0.png" width=500></p>

또한 RNN (순환 신경망)을 사용하면 가변 길이의 텍스트를 생성할 수 있습니다. 이론적으로 는 지금 예와 같은 텍스트를 출력하는 RNN도 고안할 수 있습니다. 그렇다면 [그림 10-14]와 같이 RNN을 에이전트로 이용하여 강화 학습으로 최적의 아키텍처를 찾아낼 수 있습니다.

<p align="center"><img src="./images/fig_10_14.png" width=500></p>

그림과 같이 RNN이 신경망의 아키텍처를 텍스트로 생성합니다. 이 작업이 바로 에이전트의 행동에 해당합니다. 그리고 생성된 아키텍처 (신경망)를 검증 데이터로 학습시키고 마지막으로 인식 정확도를 측정합니다. 이 인식 정확도가 보상으로 작용합니다. 논문에서는 REINFORCE 알고리즘으로 RNN의 매개변수를 갱신하였고 그 결과 RNN은 점차 인식 정확 도가 더 높은 아키텍처를 출력했습니다.

> NOTE_ 논문에서는 아키텍처의 범위를 제한하는 장치가 마련되어 있습니다. 예를 들어 합성곱 계층의 파라 미터를 필터와 스트라이드 크기만으로 제한하고, RNN으로 두 매개변수의 값을 차례대로 출력하게 하는 식 입니다.

### 10.4.4 기타 예시

심층 강화 학습을 적용한 사례는 이 외에도 많습니다. 이번 절에서는 그중 세 가지를 소개하겠 습니다.

**자율주행**

자율주행 자동차를 만들기 위한 연구가 전 세계적으로 앞다투어 이루어지고 있습니다. 대부분 은 딥러닝을 활용한 '지도학습'으로 진행되고 있으나 한편으로는 강화 학습으로 접근하는 움직임도 보입니다. [41] 자율주행이라는 문제는 환경을 관찰하여 최적의 행동(브레이크 밟기, 핸들 돌리기 등)을 선택하는 일련의 결정이기 때문에 강화 학습에 잘 들어맞습니다.

강화 학습으로 자율주행에 도전해볼 수 있는 환경으로 AWS DeepRacer[42]가 있습니다. DeepRacer는 딥러닝 기법으로 제어하는 자율주행 레이싱카입니다. 사용자는 딥러닝 알고리즘을 구현하고 보상 함수 등도 직접 설정합니다. 그런 다음 시뮬레이터를 써서 AWS에서 학습 시킵니다. 최종적으로 만들어진 모델은 가상 레이스나 실물 자동차 레이스에서 이용할 수 있습 니다. 실제로 '챔피언십 컵' 등의 레이스가 정기적으로 개최되어 상금을 걸고 경쟁을 벌이고 있 습니다.

**건물 에너지 관리**

건물은 많은 전력을 소비합니다. 그래서 전력 소비 줄이기가 중요한 과제이지만 건물 내 환경
은 매우 복잡하여 효율적으로 운영하기가 쉽지 않습니다. 이 문제를 해결하기 위해 최근 다양 한 심층 강화 학습 기법이 제안되어 좋은 성과를 거두고 있습니다. [43]

예를 들어 오피스 빌딩의 공조 설비 운영에 DQN 기반 기법이 제안되었습니다. [44] 이 연구 에서는 근무 환경을 쾌적하게 유지하면서 비용을 30% 이상 줄였다고 합니다. 또한 구글의 데이터센터도 냉각 시스템을 머신러닝으로 제어하여 전력 소비를 40%나 줄이는 데 성공했습니다. [45] 여기에도 강화 학습 기법이 일부 사용되었습니다.

**반도체 칩 설계**

반도체 칩 설계는 작은 기판에 수십억 개 이상의 트랜지스터를 배치하는 일입니다. 복잡한 문제인 만큼 많은 어려움이 따릅니다. 지금까지의 반도체 칩은 숙련된 전문가들이 오랜 시간을 투자해 설계했지만 구글은 딥러닝을 활용한 칩 설계 방법을 새롭게 제안했습니다.[46]

구글의 연구는 반도체 칩 설계를 강화 학습 문제로 간주하고, 에이전트가 다양한 반도체 레이 아웃을 시도하며 최적의 설계를 찾습니다. 그 결과, 놀랍게도 6시간 안에 설계를 완료했으며 모든 주요 지표에서 인간 전문가의 설계보다 우수하거나 동등한 성능을 달성했다고 합니다. 이 기술은 이미 구글이 개발하는 TPU Tensor Processing Unit 설계에 활용되고 있습니다.*

## 10.5 심층 강화 학습이 풀어야 할 숙제와 가능성

이번 절에서는 심층 강화 학습을 현실 세계 시스템에 적용하기 위해 풀어야 할 숙제와 해결책을 제시합니다. 그런 다음 강화 학습 문제를 MDP로 기술할 때 고려할 사항을 논의하고, 마지막으로 강화 학습의 잠재력을 이야기하며 마무리하겠습니다.

### 10.5.1 현실 세계에 적용하기

심층 강화 학습은 로봇 제어와 같은 실제 시스템에 활용되고 있습니다. 하지만 현실 세계는 가상의 세계와 달리 제약이 많습니다. 예를 들어 로봇은 비싸고 대량으로 준비하기 어렵기 때문에 경험 데이터를 충분히 수집하기가 쉽지 않습니다. 또한 강화 학습에서는 시행착오를 거치면서 경험 데이터를 수집하는데, 이 과정에서 로봇을 고장내거나 주변에 위험을 초래하는 행동은 피해야 합니다. 이번 절에서는 이러한 문제들을 막아주는 유력한 해결책을 몇 가지 소개하겠습 니다.

**시뮬레이터 활용**

현실 세계의 제약 상당수는 시뮬레이터로 해결할 수 있습니다. 시뮬레이터 안에서는 에이전트 와 환경의 상호작용을 빠르게 반복할 수 있습니다. 게다가 에이전트가 위험한 행동을 하더라도 아무런 문제가 없습니다. 하지만 현실 세계와 시뮬레이터 속 세계 사이에는 간극이 있습니다. 시뮬레이터는 실제 환경을 완벽하게 모방할 수 없기 때문에 시뮬레이터에서 학습한 모델(에이전트)이 실제 환경에서 기대한 대로 동작하지 않는 경우가 많습니다. 이 문제에 집중하는 연구 분야를 Sim2Real이라고 합니다.

Sim2Real에는 여러 가지 유력한 기법이 있으나 그중 Domain Randomization (도메인 무작위화) 기법을 소개하겠습니다. 이 기법은 시뮬레이터에 무작위 요소를 추가하여 다양한 환경을 만들고, 그 다양한 환경 속에서 에이전트를 행동시키고 학습시킵니다. 이런 식으로 일반화 성능을 높여 실제 환경에서도 더 잘 동작하리라 기대할 수 있습니다. 예를 들어 OpenAI는 손 가락 다섯 개가 달린 로봇 손 하나만으로 루빅스 큐브를 맞추는 문제에 이 기법을 적용한 연구 [47]를 공개했습니다.

<p align="center"><img src="./images/fig_10_15.png" width=500></p>

이 연구에서는 광원과 텍스처 등의 렌더링 이미지 매개변수나 마찰과 같은 역학적 매개변수에 무작위 값을 넣어 다양한 환경을 만들고 그 안에서 에이전트를 학습시킵니다. 그 결과 현실 세 계에서도 추가 학습 없이 큐브를 맞추는 데 성공했습니다.

**오프라인 강화 학습**

과거에 수집한 경험 데이터를 잘 활용하는 방법도 있습니다. 예를 들어 자율주행이나 로봇 제어 등에서는 지금까지 사람이 조작한 많은 경험 데이터가 축적되어 있을 수 있습니다. 대화 시스템에서는 사람들이 주고받은 대화 이력을 쉽게 수집할 수 있습니다(챗봇 같은 대화 시스템 을 강화 학습으로 구현하는 연구는 많습니다). 이러한 과거 경험 데이터, 즉 '오프라인 데이터'를 활용해서도 에이전트를 학습시킬 수 있습니다. 나아가 환경과 상호작용을 전혀 하지 않고도 오프라인 데이터만으로 최적의 정책을 추정하는 것도 생각할 수 있습니다. 이것이 바로 **오프라인 강화 학습** offline reinforcement learning입니다.

<p align="center"><img src="./images/fig_10_16.png" width=600></p>

왼쪽 그림과 같이 일반적인 강화 학습은 에이전트와 환경이 상호작용합니다('온라인' 강화 학습). 반면 오른쪽의 오프라인 강화 학습은 지금까지 수집된 경험 데이터(오프라인 데이터셋) 만으로 학습합니다. 오프라인 강화 학습은 실제 환경과의 상호작용이 일어나지 않는 것이 특징 입니다.

> NOTE_ 오프라인 강화 학습은 오프-정책 기법과 관련이 있습니다. 오프-정책은 '행동 정책'을 통해 경험 데 이터를 수집하고, 그 경험 데이터를 이용해 '대상 정책'을 갱신합니다. 그러므로 오프-정책 기법을 활용하여 오프라인 강화 학습을 구현할 수 있습니다. 다만 일반적인 오프-정책 기법은 대상 정책에 준하는 행동 정책을 만들고 그 위에서 환경과 상호작용하면서 에이전트를 학습시킵니다(Q 러닝을 떠올려보세요). 반면, 오프 라인 강화 학습은 환경과의 상호작용이 전혀 없이 오프라인 경험 데이터만으로 학습합니다.

오프라인 강화 학습에는 기술적으로 풀어야 할 숙제가 몇 가지 있습니다. 예를 들어 오프라인 데이터셋만으로 어떻게 정책을 평가하고 개선할 것인가 하는 숙제를 풀어야 합니다. 물론 오 프-정책 기법에서 설명한 방법도 활용할 수 있습니다(중요도 샘플링 등). 그 외에도 다양한 기법이 제안되어 현재 활발히 연구중입니다.[48]

**모방 학습**

인간은 숙련된 전문가에게서 배우는 일이 많습니다. 예를 들어 야구 선수를 꿈꾸는 소년은 프로 선수가 배트를 휘두르는 모습을 흉내 내기 위해 연습합니다. 이와 마찬가지로 전문가의 동작을 모방하도록 정책을 학습하는 방법을 생각할 수 있습니다. 이것이 바로 모방 학습 imitation learning입니다. 모방 학습에서는 전문가의 시연을 참고하여 그 동작을 모방하는 것을 목표로 정책을 학습합니다.

모방 학습에도 다양한 기법이 제안되고 있습니다. 예를 들어 「Deep Q-Learning from Demonstrations」 논문[49]에서는 인간 전문가가 아타리를 플레이하는 모습을 모방합니다. 구체적으로는 전문가의 플레이에서 얻은 '상태, 행동, 보상'의 시계열 데이터 (전문가 데이터)를 DQN의 경험 재생 버퍼에 추가하고 그 위에서 DQN으로 학습을 수행합니다. 경험 재생 버 에서 전문가 데이터가 선택될 확률을 높이고 동시에 전문가 데이터에 가까워지도록 DQN의 갱신식을 조정하는 것이죠. 이런 식으로 전문가 데이터에 가깝게 행동하는 DQN을 구현할 수 있습니다.

### 10.5.2 MDP로 공식화하기 위한 팁

강화 학습의 많은 이론은 마르코프 결정 과정 Markov Decision Process (MDP)을 전제로 합니다. 이 책에서는 강화 학습 문제가 MDP로 주어진다고 전제하고 그 위에서 다양한 알고리즘을 탐구 했습니다. 하지만 실제 문제를 강화 학습으로 풀려면 우선 문제를 MDP 형식에 맞는 공식으로 표현할 수 있어야 합니다. 그리고 MDP로 어떻게 공식화하느냐가 최종 결과에 큰 영향을 줍니 다. 이번 절에서는 강화 학습 문제를 MDP로 공식화할 때 고려할 점들을 이야기합니다.

**MDP가 지닌 유연성**

현실에 존재하는 모든 문제를 MDP라는 문제 형식에 맞출 수는 없습니다. 하지만 놀랍도록 많은 문제가 MDP로 공식화할 수 있다고 알려져 있습니다. MDP는 환경과 에이전트가 상태, 행동, 보상'이라는 세 가지 정보를 서로 던져주는 구조입니다. 이때 어떤 센서를 사용할지, 행동을 어떻게 제어할지, 보상을 어떻게 설정할지 등 세부 정보는 문제에 따라 유연하게 결정할 수 있습니다. 
이러한 유연성 덕분에 MDP를 적용할 수 있는 범위는 생각보다 넓습니다.

또한 에이전트의 행동은 고차원적인 것부터 저차원적인 것까지 생각할 수 있습니다. 예를 들어 로봇 제어 문제에서는 '쓰레기 버리기'나 '충전하기' 같은 행동을 고차원적인 결정이라고 생각할 수 있습니다. 한편 '모터에 볼트 전류를 흘려보내기'와 같은 행동은 저차원적인 결정이라고 할 수 있습니다. 상태도 마찬가지로 높은 수준부터 낮은 수준까지 다양한 관점에서 생각할 수 있 습니다.

시간(타임 스텝)의 단위도 에이전트가 결정하는 시점에 따라 다르게 설정할 수 있습니다. 예를 들어 단계 하나는 1밀리초가 될 수도 있고, 1분이 될 수도, 1시간이나 하루, 심지어 한 달이 될 수도 있습니다. 이처럼 문제에 따라 상태, 행동, 시간 단위를 유연하게 바꿀 수 있기 때문에 MDP의 적용 범위가 그만큼 넓어집니다.

**MDP에서 설정이 필요한 사항**

현실의 문제를 해결하려면 어떻게 MDP로 공식화하느냐가 관건입니다. 새로운 문제를 MDP로 공식화하려면 다음과 같은 사항들을 결정해야 합니다.

- 해결하고자 하는 문제는 일회성 과제인가, 아니면 지속적 과제인가?
- 보상의 가치는? (보상 함수 설정)
- 에이전트가 취할 수 있는 행동은 무엇인가?
- 환경의 상태를 무엇으로 규정할 것인가?
- 수익의 할인율은?
- 어디까지를 환경으로, 어디까지를 에이전트로 할 것인가?

이 사항들은 문제에 따라 자연스럽게 결정되는 경우도 있고 설정하기 어려운 경우도 있습니다. 예를 들어 바둑과 같은 보드 게임은 간단합니다. 일회성 과제이고 보상은 최종적으로 이기면 1, 지면 -1, 그 외의 수에는 0으로 설정할 수 있습니다. 에이전트가 취하는 행동은 돌을 어디에 둘 것인가가 됩니다. 환경의 상태는 바둑판 위의 돌 배치가 될 것입니다.

한편, 에이전트와 환경의 경계를 나누기가 의외로 어려운 경우가 있습니다. 일반적으로는 원하는 대로 제어할 수 있는 대상을 에이전트로 간주합니다. 로봇 팔을 조작하는 문제를 생각해봅 시다. 로봇 팔에는 관절마다 모터가 달려 있고 모터에 흐르는 전류(혹은 전압)를 자유롭게 제 어할 수 있습니다. 반면, 손이나 발과 같은 물리적 부분(하드웨어)은 마음대로 제어하기 어렵습니다. 그래서 하드웨어는 에이전트의 외부, 즉 환경으로 설정하는 게 좋습니다. 이처럼 모든 요소를 마음대로 제어할 수 있는 대상을 에이전트로 규정하는 것이 일반적입니다.

> NOTE_ 강화 학습으로 현실 세계의 문제에 도전하다 보면, 보상 함수 설계가 문제 해결의 열쇠가 되는 경우 가 많습니다. 그래서 실무에서는 현재 보상 함수의 설계와 학습된 정책의 결과를 관찰하면서 보상 함수를 반 복해서 재설계하곤 합니다.

**공식화를 잘하기 위한 팁**

MDP로 멋지게 공식화하는 방법은 과학보다는 예술에 가까운 측면이 있습니다. 이론도 중요 하지만 경험과 직관이 큰 도움이 되는 이유입니다. 그래서 특히 강화 학습 분야에서 유명한 기존 연구를 많이 살펴보라고 강력히 권합니다. 논문을 읽을 때 어떻게 MDP로 공식화했는지를 주의 깊게 살펴보세요. 그러면 멋지게 공식화하는 힌트를 많이 얻을 것입니다.

### 10.5.3 범용 인공지능 시스템

인공지능 분야는 빠르게 발전하고 있습니다. 밤낮으로 다양한 시스템이 개발되어 큰 성과를 거 두고 있죠. 하지만 거의 모든 시스템이 특정한 작업에 국한된 것도 사실입니다. 이미지를 인식하고, 바둑을 두고, 자동차를 운전하는 등의 시스템은 그 작업에만 특화되어 개발되고 있습니 다. 반면 우리 인간은 수많은 일을 할 줄 압니다. 주변 환경을 인식하고, 언어를 이해하고, 자동 차를 운전하고, 바둑을 둘 수 있습니다. 마치 인간처럼 다방면으로 활용할 수 있는 인공지능을 **범용 인공지능** Artificial General Intelligence (AGI)이라고 합니다.

강화 학습은 범용 인공지능을 실현하기 위한, 적어도 이에 근접하기 위한 핵심 분야라고 믿는 연구자가 많습니다. 예를 들어  딥마인드 DeepMind 연구진이 발표한 「Reward is Enough」논문 [50]에서는 "지능 혹은 지능과 관련한 능력은 보상 총합의 극대화만으로 충분히 이해할 수 있다"라는 가설을 논의합니다. 나아가 보상 총합의 극대화를 목표로 하는 강화 학습 기술로 범용 인공지능을 실현할 가능성을 언급하고 있습니다.

우리는 이 책을 통해 강화 학습이 무엇인지 알아봤습니다. 강화 학습은 보상 총합을 극대화한 다는 목표를 향해, 환경과의 상호작용 속에서 에이전트가 정책을 학습합니다. '보상 총합 극대화'라는 목적만 있다면 어쩌면 지적 능력을 갖추기에 충분할지도 모릅니다. 그리고 강화 학습 을 활용한 기법이 향후 범용 인공지능을 구현해 낼지도 모릅니다. 물론 아직까지는 가설일 뿐 이라서 현재로서는 답을 알 수 없습니다. 하지만 이런 관점이 있다는 사실을 아는 것만으로도 세상을 바라보는 시야가 넓어지지 않을까 생각합니다.

## 10.6 정리

이번 장에서는 심층 강화 학습 알고리즘들을 어떻게 분류할 수 있는지 알아보았습니다. 그런 다음 발전된 심층 강화 학습 알고리즘의 예로 정책 경사법 계열과 DQN 계열의 유명한 알고리 즘들을 살펴봤습니다. 또한 심층 강화 학습에서 중요한 연구 사례들을 몇 가지 소개했습니다. 이 사례들만 보더라도 딥러닝의 영향력이 얼마나 큰지 실감할 수 있으리라 생각합니다.

지금까지 강화 학습에 대해 많은 것을 배웠습니다. 이 책의 전반부에서는 강화 학습의 기초를 배웠고 후반부에서는 현재 통용되는 심층 강화 학습 기술들을 배웠습니다. 최첨단 심층 강화 학 습들도 옛날부터 쌓아온 기초 위에 세워졌습니다. 그래서 이 책에서 지금까지 익힌 지식은 여러 분이 한층 진보한 심층 강화 학습의 세계로 나아가는 데 훌륭한 밑거름이 되어줄 것입니다.